## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [33]:
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()

cnx = mysql.connector.connect(
    host=os.getenv("HOSTNAME"),
    user=os.getenv("USERNAME"),
    password=os.getenv("PASSWORD")
)

print(cnx)

Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [21]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [3]:
cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")

In [4]:
cursor.execute("SHOW DATABASES;")

for db in cursor:
    print(db)

('aneis_nodeb',)
('dbprodutos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('trafego_nodeb',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [5]:
import pandas as pd

df_livros = pd.read_csv("/home/jessica/pipeline-python-mongo-mysql/data/tabela_livros.csv")
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,68536309141e99c09eeb4d1e,Modelagem preditiva,livros,70.49,1.337820,2023-02-01,Isabella Pereira,PE,4,cupom,1,-8.38,-37.86
1,68536309141e99c09eeb4da1,Modelagem preditiva,livros,105.99,3.228088,2022-07-01,Thiago Silva,SP,1,cartao_credito,4,-22.19,-48.79
2,68536309141e99c09eeb4eb0,Dashboards com Power BI,livros,47.26,0.000000,2021-05-02,Isabella Pereira,RJ,2,cartao_credito,10,-22.25,-42.66
3,68536309141e99c09eeb4ce6,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
4,68536309141e99c09eeb4d16,Dashboards com Power BI,livros,41.76,2.754699,2022-02-01,Juliana Costa,SP,5,cartao_credito,1,-22.19,-48.79


In [6]:
df_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [7]:
df_livros.shape

(742, 13)

In [8]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliação_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qtde_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               
               PRIMARY KEY (id)
    );             
""")

**Selecionando a base de dados para verificar a tabela criada**

In [9]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
    print(tb)

('tb_livros',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [10]:
for i, row in df_livros.iterrows():
    print(tuple(row))

('68536309141e99c09eeb4d1e', 'Modelagem preditiva', 'livros', 70.49, 1.337819504, '2023-02-01', 'Isabella Pereira', 'PE', 4, 'cupom', 1, -8.38, -37.86)
('68536309141e99c09eeb4da1', 'Modelagem preditiva', 'livros', 105.99, 3.2280877057, '2022-07-01', 'Thiago Silva', 'SP', 1, 'cartao_credito', 4, -22.19, -48.79)
('68536309141e99c09eeb4eb0', 'Dashboards com Power BI', 'livros', 47.26, 0.0, '2021-05-02', 'Isabella Pereira', 'RJ', 2, 'cartao_credito', 10, -22.25, -42.66)
('68536309141e99c09eeb4ce6', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('68536309141e99c09eeb4d16', 'Dashboards com Power BI', 'livros', 41.76, 2.7546988942, '2022-02-01', 'Juliana Costa', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('68536309141e99c09eeb4d50', 'Dashboards com Power BI', 'livros', 40.85, 0.0, '2021-04-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('68536309141e99c09eeb4db6', 'Modelagem preditiv

In [11]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]
lista_dados

[('68536309141e99c09eeb4d1e',
  'Modelagem preditiva',
  'livros',
  70.49,
  1.337819504,
  '2023-02-01',
  'Isabella Pereira',
  'PE',
  4,
  'cupom',
  1,
  -8.38,
  -37.86),
 ('68536309141e99c09eeb4da1',
  'Modelagem preditiva',
  'livros',
  105.99,
  3.2280877057,
  '2022-07-01',
  'Thiago Silva',
  'SP',
  1,
  'cartao_credito',
  4,
  -22.19,
  -48.79),
 ('68536309141e99c09eeb4eb0',
  'Dashboards com Power BI',
  'livros',
  47.26,
  0.0,
  '2021-05-02',
  'Isabella Pereira',
  'RJ',
  2,
  'cartao_credito',
  10,
  -22.25,
  -42.66),
 ('68536309141e99c09eeb4ce6',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('68536309141e99c09eeb4d16',
  'Dashboards com Power BI',
  'livros',
  41.76,
  2.7546988942,
  '2022-02-01',
  'Juliana Costa',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('68536309141e99c09eeb4d50',
  'Dashboards com Power BI',
  'livros',
  40.

In [12]:
sql = "INSERT INTO dbprodutos.tb_livros VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

cursor.executemany(sql, lista_dados)
cnx.commit()

IntegrityError: 1062 (23000): Duplicate entry '68536309141e99c09eeb4d1e' for key 'tb_livros.PRIMARY'

In [ ]:
print(cursor.rowcount, "dados inseridos")

742 dados inseridos


## Visualizando os dados inseridos

In [ ]:
cursor.execute("SELECT * FROM dbprodutos.tb_livros;")

for row in cursor:
    print(row)

('68536309141e99c09eeb4ce5', 'Modelagem preditiva', 'livros', 92.45, 5.61, datetime.date(2020, 1, 1), 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('68536309141e99c09eeb4ce6', 'Iniciando em programação', 'livros', 43.84, 0.0, datetime.date(2020, 1, 1), 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('68536309141e99c09eeb4cf8', 'Iniciando em programação', 'livros', 63.25, 3.89, datetime.date(2022, 1, 1), 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('68536309141e99c09eeb4d06', 'Ciência de dados com python', 'livros', 86.13, 5.27, datetime.date(2021, 2, 1), 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('68536309141e99c09eeb4d08', 'Ciência de dados com python', 'livros', 72.75, 1.46, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('68536309141e99c09eeb4d0b', 'Iniciando em programação', 'livros', 61.73, 0.0, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'SP', 4, 'cartao_credi

In [15]:
import pandas as pd

df_livros2021 = pd.read_csv("/home/jessica/pipeline-python-mongo-mysql/data/tabela_2021_em_diante.csv")
df_livros2021.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,68536309141e99c09eeb4cf1,Tablet ABXY,eletronicos,1100.42,68.451348,2021-01-01,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
1,68536309141e99c09eeb4cfc,Boneca bebê,brinquedos,50.72,3.231792,2023-01-01,Isabella Pereira,SP,4,boleto,1,-22.19,-48.79
2,68536309141e99c09eeb4d1b,Fone de ouvido,eletronicos,108.12,6.696913,2023-02-01,Beatriz Moraes,PR,4,cartao_credito,5,-24.89,-51.55
3,68536309141e99c09eeb4d29,Geladeira,eletrodomesticos,2774.00,147.990281,2021-03-01,Bruno Rodrigues,RJ,5,cartao_credito,1,-22.25,-42.66
4,68536309141e99c09eeb4d32,Bola de vôlei,esporte e lazer,54.42,3.180827,2022-03-01,Lucas Oliveira,SP,5,cartao_credito,1,-22.19,-48.79


In [17]:
df_livros2021.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [18]:
df_livros2021.shape

(6574, 13)

In [19]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros_2021_em_diante(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliação_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qtde_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               
               PRIMARY KEY (id)
    );             
""")

In [22]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
    print(tb)

('tb_livros',)
('tb_livros_2021_em_diante',)


In [23]:
for i, row in df_livros2021.iterrows():
    print(tuple(row))

('68536309141e99c09eeb4cf1', 'Tablet ABXY', 'eletronicos', 1100.42, 68.4513476183, '2021-01-01', 'Beatriz Moraes', 'SP', 1, 'cupom', 1, -22.19, -48.79)
('68536309141e99c09eeb4cfc', 'Boneca bebê', 'brinquedos', 50.72, 3.2317919389, '2023-01-01', 'Isabella Pereira', 'SP', 4, 'boleto', 1, -22.19, -48.79)
('68536309141e99c09eeb4d1b', 'Fone de ouvido', 'eletronicos', 108.12, 6.6969125635, '2023-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 5, -24.89, -51.55)
('68536309141e99c09eeb4d29', 'Geladeira', 'eletrodomesticos', 2774.0, 147.9902805969, '2021-03-01', 'Bruno Rodrigues', 'RJ', 5, 'cartao_credito', 1, -22.25, -42.66)
('68536309141e99c09eeb4d32', 'Bola de vôlei', 'esporte e lazer', 54.42, 3.1808271914, '2022-03-01', 'Lucas Oliveira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('68536309141e99c09eeb4d34', 'Lavadora de roupas', 'eletrodomesticos', 2060.2, 107.2838911531, '2022-03-01', 'Mariana Ferreira', 'PB', 4, 'cartao_credito', 5, -7.28, -36.72)
('68536309141e99c09eeb4d35', 'Ca

In [25]:
lista_dados2021 = [tuple(row) for i, row in df_livros2021.iterrows()]
lista_dados2021

[('68536309141e99c09eeb4cf1',
  'Tablet ABXY',
  'eletronicos',
  1100.42,
  68.4513476183,
  '2021-01-01',
  'Beatriz Moraes',
  'SP',
  1,
  'cupom',
  1,
  -22.19,
  -48.79),
 ('68536309141e99c09eeb4cfc',
  'Boneca bebê',
  'brinquedos',
  50.72,
  3.2317919389,
  '2023-01-01',
  'Isabella Pereira',
  'SP',
  4,
  'boleto',
  1,
  -22.19,
  -48.79),
 ('68536309141e99c09eeb4d1b',
  'Fone de ouvido',
  'eletronicos',
  108.12,
  6.6969125635,
  '2023-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  5,
  -24.89,
  -51.55),
 ('68536309141e99c09eeb4d29',
  'Geladeira',
  'eletrodomesticos',
  2774.0,
  147.9902805969,
  '2021-03-01',
  'Bruno Rodrigues',
  'RJ',
  5,
  'cartao_credito',
  1,
  -22.25,
  -42.66),
 ('68536309141e99c09eeb4d32',
  'Bola de vôlei',
  'esporte e lazer',
  54.42,
  3.1808271914,
  '2022-03-01',
  'Lucas Oliveira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('68536309141e99c09eeb4d34',
  'Lavadora de roupas',
  'eletrodomesticos',


In [26]:
sql = "INSERT INTO dbprodutos.tb_livros_2021_em_diante VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

cursor.executemany(sql, lista_dados2021)
cnx.commit()

In [27]:
print(cursor.rowcount, "dados inseridos")

6574 dados inseridos


In [29]:
cursor.execute("SELECT * FROM dbprodutos.tb_livros_2021_em_diante;")

for row in cursor:
    print(row)

('68536309141e99c09eeb4cee', 'Xadrez de madeira', 'brinquedos', 25.23, 0.0, datetime.date(2021, 1, 1), 'Thiago Silva', 'BA', 5, 'cartao_credito', 2, -13.29, -41.71)
('68536309141e99c09eeb4cef', 'Impressora', 'eletronicos', 322.04, 14.73, datetime.date(2021, 1, 1), 'João Souza', 'SP', 3, 'cartao_credito', 1, -22.19, -48.79)
('68536309141e99c09eeb4cf0', 'Mesa de centro', 'moveis', 282.22, 12.61, datetime.date(2021, 1, 1), 'João Souza', 'SP', 3, 'boleto', 1, -22.19, -48.79)
('68536309141e99c09eeb4cf1', 'Tablet ABXY', 'eletronicos', 1100.42, 68.45, datetime.date(2021, 1, 1), 'Beatriz Moraes', 'SP', 1, 'cupom', 1, -22.19, -48.79)
('68536309141e99c09eeb4cf2', 'Fogão', 'eletrodomesticos', 791.81, 42.44, datetime.date(2021, 1, 1), 'Juliana Costa', 'SP', 4, 'boleto', 1, -22.19, -48.79)
('68536309141e99c09eeb4cf3', 'Violão', 'instrumentos musicais', 316.84, 17.16, datetime.date(2022, 1, 1), 'Pedro Gomes', 'SP', 5, 'cartao_credito', 2, -22.19, -48.79)
('68536309141e99c09eeb4cf4', 'Geladeira', 'el

In [30]:
cursor.close()

True

In [31]:
cnx.close()